<a href="https://colab.research.google.com/github/yaniv496/Data-Science-Projects/blob/main/random_forest_yaniv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gdown
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import datetime


def plot_plt():
    train = pd.read_csv('train.csv')

    # Select the relevant features
    features = ['SalePrice', 'ModelID']
    X = train[features]

    # Scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(X['SalePrice'], X['MachineHoursCurrentMeter'], alpha=0.5)
    plt.title('Scatter Plot of Feature1 vs Feature2')
    plt.xlabel('Feature1')
    plt.ylabel('Feature2')
    plt.grid(True)
    plt.show()

def pair_plot():
    features = ['ModelID', 'machine_age']
    #sns.pairplot(train[features + ['SalePrice']])
    plt.show()

def RMSE(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


def base_model():
    #read train data:
    train = pd.read_csv('train.csv')
    # calculate perdicition
    baseline_prediction = train['SalePrice'].mean()

    # Create a submission file

    # read the the new data
    valid = pd.read_csv('valid.csv')

    # create new file with new data - id and the prediction value and save data to file
    submission = pd.DataFrame({'SalesID': valid['SalesID'], 'SalePrice': baseline_prediction})

    submission.to_csv('baseline_submission.csv', index=False)




def RandomForest():

    def download_from_gdrive(url, filename):
        # Extract the file ID from the URL
        file_id = url.split('/')[-2]
        download_url = f"https://drive.google.com/uc?id={file_id}"

        # Download the file
        if Path(filename).exists():
            print(f"File '{filename}' already exists. Skipping download.")
        else:
            gdown.download(download_url, filename, quiet=False)
            print(f"File downloaded as: {filename}")

    train = 'https://drive.google.com/file/d/1guqSpDv1Q7ZZjSbXMYGbrTvGns0VCyU5/view?usp=drive_link'
    valid = 'https://drive.google.com/file/d/1j7x8xhMimKbvW62D-XeDfuRyj9ia636q/view?usp=drive_link'
    download_from_gdrive(train, 'train.csv')
    download_from_gdrive(valid, 'valid.csv')


    train = pd.read_csv('train.csv')
    valid = pd.read_csv('valid.csv')
    #train.head() # inspect the first row of data
    train['saleyeardate'] = None
    valid['saleyeardate'] = None
    # #add train age features
    #
    train['saleyeardate'] = pd.to_datetime(train['saledate']).dt.year
    valid['saleyeardate'] = pd.to_datetime(valid['saledate']).dt.year

#--------------- background calc
    # calculate mean without year - > 1000 on  YearMade feature
    # Filter out the specific value
    #filtered_data = train[train['YearMade'] != 1000]

    # Calculate the mean of the filtered feature
    #mean_value = filtered_data['YearMade'].mean()
    # the result of mean is 1994 in train
    #--------------
   #filtered_data = valid[valid['YearMade'] != 1000]
    #mean_value = filtered_data['YearMade'].mean()
    # the result of mean is 1999 in valid


    # update the mean 1994 (train) 1999 (valid) while the value is 1000 in YearMade
    # train.loc[train['YearMade'] == 1000, 'YearMade'] = 1994
    valid.loc[valid['YearMade'] == 1000, 'YearMade'] = 1999



    # Drop the rows
    train_c1 = train.drop(index=train[train['YearMade'] == 1000].index)
    #valid_c1 = valid.drop(index=valid[valid['YearMade'] == 1000].index)

    valid_c1 = valid
    train_c1['machine_age'] = train_c1['saleyeardate'] - train_c1['YearMade']
    valid_c1['machine_age'] = valid_c1['saleyeardate'] - valid_c1['YearMade']

    train_c1 = train_c1.drop(index=train_c1[train_c1['machine_age'] < 0].index)
    #valid_c1 = valid_c1.drop(index=valid_c1[valid_c1['machine_age'] < 0].index)



    # clean train data
    train_c1 = train_c1.select_dtypes('number')# drop all categorical variables
    train_c1 = train_c1[train_c1['machine_age'] < 60]

    train_c1 = train_c1.dropna()  # drop all rows with missing values
    train_c1 = train_c1.set_index('SalesID')


    # Split the data
    X = train_c1.drop(columns=['SalePrice'])
    y = train_c1['SalePrice']
    #test_size - > % from tha data for testing and the other for training

    # plots
    # features_list = ['ModelID', 'machine_age']
    # sns.pairplot(train_c1[features_list + ['SalePrice']])
    # plt.show()

    # random_state - An integer value ensures that the  results are reproducible
    test_size = 0.1 # ( 25% - testing , 75% - training)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)


    # Train the model
    model = RandomForestRegressor(random_state=42,n_estimators=300, n_jobs=-1)
    model.fit(X_train, y_train)

    #print(pd.Series(model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False))


    # Validate the model
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    print(f'Train RMSE: {RMSE(y_train, y_train_pred)}')
    print(f'Test RMSE: {RMSE(y_test, y_test_pred)}')

    # Clean and prepare validation data


    #valid_c1 = valid.select_dtypes('number')  # drop all categorical variables
    #valid_c1 = valid_c1[valid_c1['machine_age'] < 40]
    valid_c1 = valid.fillna(-1)
    #valid_c1 = valid_c1.dropna()  # drop all rows with missing values
    # plots
    # features_list = ['machine_age']
    # sns.pairplot(valid_c1[features_list + ['ModelID']])
    # plt.show()

    #summission
    # Ensure the validation data has the same features as the training data
    X_valid = valid_c1.set_index('SalesID')[X.columns]
    y_valid_pred = model.predict(X_valid)
    y_valid_pred = pd.Series(y_valid_pred, index=X_valid.index, name='SalePrice')
    # Print predictions for validation data
    #print("Predictions for validation data:")
    #print(y_valid_pred)
    #print('mean avg: ' + str(y_valid_pred.mean()))

    # Create a submission file  -# add date-time to csv file name

    # timestamp = datetime.now().isoformat()
    timestamp = datetime.now().isoformat().replace(':', '_')
    filename = f'submission_{timestamp}.csv'
    y_valid_pred.to_csv(filename)
    print (pd.Series(model.feature_importances_, index=model.feature_names_in_).sort_values(ascending=False))

# Press the green button in the gutter to run the script.
if __name__ == '__main__':
   # plot_plt()
    RandomForest()

# See PyCharm help at https://www.jetbrains.com/help/pycharm/
